In [1]:
from imports import *
from PreProcessor import *
import os

In [3]:

CodeTokens = set()
UCTokens = set()
UC_documents = list()
code_documents = list()
TotalTokens=set()




for filename in os.listdir("./UC"):
    filepath = os.path.join("./UC", filename)
    tokens = UCPreProcessor(filepath)
    UC_documents.append(tokens)
    UCTokens.update(tokens.split())

print(UCTokens)
print(UC_documents)

for filename in os.listdir("./CC"):
    filepath = os.path.join("./CC", filename)
    tokens = CodePreProcessor(filepath)
    code_documents.append(tokens)
    CodeTokens.update(tokens.split())

{'he', 'issu', 'request', 'number', 'result', 'specifi', 'access', 'want', 'server', 'editnew', 'new', 'histori', "'s", 'allow', 'save', 'person', 'alreadi', 'given', 'inseriscifeedback', 'read', 'turn', 'registr', 'tourist', 'exceed', 'second', 'bookmark', 'card', 'modificapreferenzediricerca', 'receipt', 'fill', 'activeagr', 'feedbackgi', 'perform', 'button', 'connect', 'errorchangepassword', 'display', 'duplic', 'success', 'use', 'specif', 'involv', 'creat', 'check', 'previou', 'postnew', 'report', 'heritag', 'combin', 'ad', 'locat', 'modifyculturalgood', 'call', 'where', 'view', "'wa", 'area', 'time', 'imag', 'tag', 'less', 'requir', 'searchrefreshmentpoint', 'notifi', 'visualizzaschedapuntodiristoro', 'field', 'criteria', 'statist', 'incorrect', 'visit', 'menu', 'cancel', 'research', 'maximum', 'local', 'editcom', 'privileg', 'alter', 'appropri', 'shall', 'featur', 'search', 'site', 'cultur', 'researchforculturalgood', 'point', 'correspond', 'choic', 'rest', 'notic', 'deactiv', 'f